<a href="https://colab.research.google.com/github/matloob-altaf/Machine-Learning-LUMS/blob/main/Programming_Assignment_1/21100164.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
import numpy as np
import pandas as pd
import matplotlib as plt
import scipy as sc
import string
import re

In [54]:
df_train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ML-PA1/train.csv")
df_test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ML-PA1/test.csv")

df_train.head(10)

,Sentiment,Tweet
0,neutral,"@united 877 from amsterdam to ewr, 02.27.2015,..."
1,negative,@united IT-problems with the link? #3thparty
2,positive,@united -today the staff @ MSP took customer s...
3,negative,@AmericanAir but have been yet to receive assi...
4,negative,@SouthwestAir you won't let me change my rese...
5,neutral,@AmericanAir where do I look for cabin crew va...
6,neutral,@united please update what is going to happen ...
7,negative,@AmericanAir how do I redeem a travel voucher ...
8,neutral,@AmericanAir originating at SFO and going to LAX.
9,negative,@JetBlue Good perspective. If only this safety...


In [55]:
print("Train data dimentions", df_train.shape)
print("Test data dimentions", df_test.shape)


Train data dimentions (11680, 2)
Test data dimentions (2921, 2)


## Preprocessing
*removing the stop words, punctuation marks and other unwanted characters from the tweets and converting them to lower case*

In [56]:
# Loading stop words
stopWords = []
with open('/content/drive/My Drive/Colab Notebooks/ML-PA1/stop_words.txt', 'r') as fi:
    for line in fi:
        line = line.rstrip()
        stopWords.append(line)

print(stopWords)

['i', "i'm", 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 't

In [57]:
def removeUnwantedChars(tweet):
  #  Converts to lower
  result = tweet.lower()
  # Removing stop words
  result = ' '.join(filter(lambda x: x not in stopWords,  result.split()))
  # Removes numbers
  result = re.sub(r'\d+', '', result)
  # Removes punctuactions
  result = re.sub(r'[^\w\s]','', result)
  # Removes leading and trailing white spaces
  result = result.strip()

  return result

In [58]:
df_train['preprocessedTweets'] = df_train['Tweet'].apply(removeUnwantedChars)
df_train.head()

,Sentiment,Tweet,preprocessedTweets
0,neutral,"@united 877 from amsterdam to ewr, 02.27.2015,...",united amsterdam ewr
1,negative,@united IT-problems with the link? #3thparty,united itproblems link thparty
2,positive,@united -today the staff @ MSP took customer s...,united today staff msp took customer service ...
3,negative,@AmericanAir but have been yet to receive assi...,americanair yet receive assistance one agents ...
4,negative,@SouthwestAir you won't let me change my rese...,southwestair let change reservation online was...


In [59]:
df_test['preprocessedTweets'] = df_test['Tweet'].apply(removeUnwantedChars)
df_test.head()

,Sentiment,Tweet,preprocessedTweets
0,neutral,@VirginAmerica to jump into the Dallas-Austin ...,virginamerica jump dallasaustin market dallas...
1,positive,"@AmericanAir Chicago seen from seat 6A, AA 162...",americanair chicago seen seat a aa far great ...
2,negative,@united you need a bag bouncer. Get it together,united need bag bouncer get together
3,negative,"@JetBlue Hey Jetblue, you stranded an entire p...",jetblue hey jetblue stranded entire plane supp...
4,negative,@USAirways Big fail on not having curbside bag...,usairways big fail curbside baggage pittsburgh...
